# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [63]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats.distributions import norm

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [9]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [10]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [12]:
data.head(10)

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit
5,b,1,4,2,6,1,0,0,0,266,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private
6,b,1,4,2,5,0,1,0,0,13,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private
7,b,1,3,4,21,0,1,0,1,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit
8,b,1,4,3,3,0,0,0,0,316,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private
9,b,1,4,2,6,0,1,0,0,263,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private


In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

  ### Q 1. What test is appropriate for this problem? Does CLT apply?

The t-test is most appropriate when we need to determine if there is a statistically significant difference between the two independent sample groups that is not pure chance.
The sample size is large enough (n>30) so the normal distribution can be used and the Central Limit Theorem applies.

### Q 2. What are the null and alternate hypotheses?

Ho: There is no call back difference between with black sounding names and white sounding names resumes.

H1: There is a call back difference between with black sounding names and white sounding names resumes.

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Q 3. Compute margin of error, confidence interval, and p-value.

In [33]:
#splitting the dataset
white = data[data['race']=='w']
black = data[data['race']=='b']

In [76]:
#calculate the mean difference 
b_mean = black['call'].mean()
w_mean = white['call'].mean()
md = w_mean - b_mean

print('Mean difference: {} %'.format(md*100))

Mean difference: 3.2032854855060577 %


In [79]:
perc_w_call = white['call'].mean()*100
perc_b_call = black['call'].mean()*100

print('White call back: {} %'.format(perc_w_call))
print('Black call back: {} %'.format(perc_b_call))

White call back: 9.650924056768417 %
Black call back: 6.44763857126236 %


In [68]:
X = white['call']
Y = black['call']

In [80]:
#Standard Error
se = np.sqrt(np.var(X)/len(X) + np.var(Y)/len(Y))
print('Standard Error: {} '.format(se))

Standard Error: 0.007783308165446165 


In [71]:
#Lower and Upper confidence limits
lcl,ucl = md-2*se,md+2*se
lcl,ucl

(0.016466238524168248, 0.04759947118595291)

In [82]:
#Z-score
z=md/se
print('z-score: {}'.format(z))

z-score: 4.115583524916792


In [83]:
#P-value
pvalue = -2*norm.cdf(-np.abs(z))
print('p-value: {}'.format(pvalue))

p-value: -3.8620110799840656e-05


Margin of Error : z-value(95%) x standard error:

z-value for 95% confidence is 1.96 <br>
standard error has been calculated above as 0.007783308165446165 

In [86]:
print('Marigin of Error: {}'.format(1.96*se))

Marigin of Error: 0.015255284004274483


With the p-value from the test being less than <0.05 we can reject the null hypothesis

### Q 4. Write a story describing the statistical significance in the context or the original problem.
    
    
   

It appears that racial discrimination exist in the United States labor market when resume's are presented with black-sounding or white-sounding names.


###  5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

This does not necessarily mean that race is the only factor and further analysis can be done to see if there was any influence on years of experience or type of employer and other factors.